In [26]:
import os
import sys
sys.path.append('/scratch/programming/preprocessing-pipeline/src')
import numpy as np
import shutil
from skimage import io
from tqdm import tqdm
from atlas.NgSegmentMaker import NgConverter
from lib.sqlcontroller import SqlController
from lib.file_location import FileLocationManager
from lib.utilities_process import test_dir, SCALING_FACTOR
from lib.utilities_mask import place_image
import scipy
import cv2

In [11]:
animal = 'DK55'
sqlController = SqlController(animal)
fileLocationManager = FileLocationManager(animal)
INPUT = fileLocationManager.aligned_rotated_and_padded_thumbnail_mask
error = test_dir(animal, INPUT, downsample=True, same_size=True)
if len(error) > 0:
    print(error)
    sys.exit()

OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'shell')
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

files = sorted(os.listdir(INPUT))
len_files = len(files)
midpoint = len_files // 2
resolution = sqlController.scan_run.resolution
resolution = int(resolution * 1000 / SCALING_FACTOR)


In [42]:
volume = []
for file in tqdm(files):
    tif = io.imread(os.path.join(INPUT, file))
    nseg,map,stat,position = cv2.connectedComponentsWithStats(tif)
    maskid = np.argmax(stat[1:,-1])+1
    mask_position = position[maskid]
    mask = map[mask_position[0],mask_position[1]]
    volume.append(mask)
ids = np.unique(volume)
ids = [(i,i) for i in ids]
volume = np.array(volume).astype('uint8')
volume = np.swapaxes(volume, 0, 2)


  0%|          | 0/485 [00:00<?, ?it/s]<ipython-input-42-0da203702b67>:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mask = map[[position[maskid]]]
  0%|          | 0/485 [00:00<?, ?it/s]


IndexError: arrays used as indices must be of integer (or boolean) type

In [43]:
map.shape

(1546, 2359)

In [39]:
position[6]

array([1167.07644353,  723.73747797])

In [31]:
nseg,map,stat,position = cv2.connectedComponentsWithStats(tif)
maskid = np.argmax(stat[1:,-1])+1

In [37]:
np.argmax(stat[1:,-1])+1

6

In [25]:
# fvolume = scipy.ndimage.gaussian_filter(volume,sigma=1.5)
fvolume = (fvolume!=0).astype('uint8')
fvolume = np.pad(fvolume,[[10,10],[10,10],[10,10]])
ng = NgConverter(fvolume, [resolution, resolution, 20000], offset=[0,0,0])
ng.create_neuroglancer_files(OUTPUT_DIR,ids)

Uploading: 100%|██████████| 7600/7600 [07:10<00:00, 17.67it/s]


Volume Bounds:  Bbox([0, 0, 0],[2379, 1566, 505], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[2379, 1566, 505], dtype=int32)


Tasks: 100%|██████████| 999/999 [00:06<00:00, 162.19it/s]


In [8]:
volume.shape

(2359, 1546, 485)